In [ ]:
import networkx as nx
import math
import numpy as np

def load_seed(s1, s2, s_pair):
  Graph1 = nx.read_edgelist(s1)
  Graph2 = nx.read_edgelist(s2)
  with open(s_pair,'r') as seed_pair_:
    lines = seed_pair_.readlines()
  return Graph1,Graph2,lines

def load_pair(lines):
  graph1p = []
  graph1pdir = {}
  graph2p = []
  for i in lines:
    i = i.strip('\n')
    i = i.split(' ')
    graph1p.append(i[0])
    graph2p.append(i[1])
    graph1pdir[i[0]] = i[1]
  return graph1p,graph2p,graph1pdir

def get_unpair(Graph1,Graph2,graph1p,graph2p):
  graph1np=[]
  graph2np=[]
  for i in Graph1.nodes:
    if i not in graph1p:
      graph1np.append(i)
  for k in Graph2.nodes:
    if k not in graph2p:
      graph2np.append(j)
  return graph1np,graph2np

In [ ]:
def load_degree(node_list, G):
  degree_direct = {}
  for j in node_list:
    degree_direct[j] = G.degree(j)
  return degree_direct

def eval_in_pairs(node, G, pair_list):
  in_pair_list = []
  for neighbor in G.neighbors(node):
    if neighbor in pair_list:
      in_pair_list.append(neighbor)
  return in_pair_list

def eval_in_other_neighbor(neighbor_pair_list,other_G,pair_dir,other_node):
  other_pair_list=[]
  for i in neighbor_pair_list:
    other_pair_list.append(pair_dir[i])
  count = 0
  for j in other_pair_list:
    if j in other_G.neighbors(other_node):
      count += 1
  return count

In [ ]:
def load_score(node1_degree, node2_degree, cnt):
  score = cnt/((math.sqrt(node1_degree))*(math.sqrt(node2_degree)))
  return score


def load_ECCE(score_list):
  std = np.std(score_list)
  max1 = max(score_list)
  score_list.remove(max1)
  max2 = max(score_list)
  if std == 0:
    result = 0
  else:
    result = (max1-max2)/std
  return result

def load_key(dict, value):
  for k, v in dict.items():
    if v == value:
      a=k
  return a

In [ ]:
def G1_node_compare_G2(node,G1,G1_pair,G2,G1_pair_dir,G1_unpair_degree_dir,G2_unpair_degree_dir,G2_unpair):
  node_pair_list = eval_in_pairs(node,G1,G1_pair)
  G2_node_and_score = {}
  for j in G2_unpair:
    count = eval_in_other_neighbor(node_pair_list,G2,G1_pair_dir,j)
    score = load_score(G1_unpair_degree_dir[node],G2_unpair_degree_dir[j],count)
    G2_node_and_score[j] = score

  score_list = list(G2_node_and_score.values())
  max_score_G2_node = load_key(G2_node_and_score,max(score_list))
  ECCE_score = load_ECCE(score_list)
  return ECCE_score, max_score_G2_node

def update_pair(ecce,node1,node2,v,G1_pair,G2_pair,G1_pair_dir,G1_unpair,G2_unpair):
  if ecce > v:
    G1_pair.append(node1)
    G2_pair.append(node2)
    G1_pair_dir[node1] = node2
    G1_unpair.remove(node1)
    G2_unpair.remove(node2)

  return G1_pair,G2_pair,G1_pair_dir,G1_unpair,G2_unpair


def load_result(v, G1, G2, G1_pair, G2_pair, G1_pair_dir, G1_unpair, G2_unpair, G1_unpair_degree_dir, G2_unpair_degree_dir):
    updated = True  
    while updated:
        updated = False  
        for node in G1_unpair:
            ecce, max_node = G1_node_compare_G2(node, G1, G1_pair, G2, G1_pair_dir, G1_unpair_degree_dir, G2_unpair_degree_dir, G2_unpair)
            G1_pair, G2_pair, G1_pair_dir, G1_unpair, G2_unpair = update_pair(ecce, node, max_node, v, G1_pair, G2_pair, G1_pair_dir, G1_unpair, G2_unpair)
            if ecce > v:  
                updated = True

    return G1_pair, G2_pair, G1_pair_dir, G1_unpair, G2_unpair

def running(seed1,seed2,seed_pair,v):
  G1,G2,node_pairs = load_seed(seed1,seed2,seed_pair)
  old_G1_pair,old_G2_pair,old_G1_pair_dir = load_pair(node_pairs)

  G1_pair = []
  G2_pair = []
  G1_pair_dir = {}
  for i in range(2777):
    G1_pair.append(old_G1_pair[i])
    G2_pair.append(old_G2_pair[i])
    G1_pair_dir[old_G1_pair[i]] = old_G2_pair[i]
  G1_unpair,G2_unpair = get_unpair(G1,G2,G1_pair,G2_pair)
  G1_unpair_degree_dir = load_degree(G1_unpair,G1)
  G2_unpair_degree_dir = load_degree(G2_unpair,G2)

  G1_pair,G2_pair,G1_pair_dir,G1_unpair,G2_unpair = load_result(v,G1,G2,G1_pair,G2_pair,G1_pair_dir,G1_unpair,G2_unpair,G1_unpair_degree_dir,G2_unpair_degree_dir)
  return G1_pair,G2_pair,G1_pair_dir,G1_unpair,G2_unpair,old_G1_pair,old_G2_pair,old_G1_pair_dir

In [ ]:
from google.colab import files

uploaded_G1 = files.upload()
uploaded_G2 = files.upload()
uploaded_seed_pairs = files.upload()

s1 = 'G1.edgelist'
s2 = 'G2.edgelist'
s_pair = 'seed_mapping.txt'
v = 0.5
G1_pair, G2_pair, G1_pair_dir, G1_unpair, G2_unpair, old_G1_pair, old_G2_pair, old_G1_pair_dir = running(s1, s2, s_pair, v)
count = 0

# Changing the loop range to use the length of old_G1_pair
for i in range(len(old_G1_pair)):
    # Checking if the item is in G1_pair and G1_pair_dir matches old_G1_pair_dir
    if old_G1_pair[i] in G1_pair and G1_pair_dir[old_G1_pair[i]] == old_G1_pair_dir[old_G1_pair[i]]:
        count += 1

# Accuracy calculation
accuracy = count / len(old_G1_pair)
print('The accuracy is: ' + str(accuracy * 100) + '%')  # Multiply by 100 to get the percentage

print(len(G1_pair))
print(len(G1_unpair))


with open('file_pairs.txt', 'w')as f:
  for node, direction in zip(G1_pair, G1_pair_dir.values()):
    s = f"{node} {direction}\n"
    f.write(s)

files.download('file_pairs.txt')

Saving G1.edgelist to G1.edgelist


Saving G2.edgelist to G2.edgelist


Saving seed_mapping.txt to seed_mapping.txt
24965
1991


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>